In [1]:
from cambrian import Dataset
import h5py
import json
import numpy as np
import cv2
import os

In [2]:
width = 720
height = 1280
dt = h5py.special_dtype(vlen=str)
h5 = h5py.File('test2.h5', 'w')
dataset = ['train', 'validate']

train_num = len(os.listdir('train_gts'))
validate_num = len(os.listdir('test_gts'))
# train_num = 1
# validate_num = 1

h5.create_dataset("meta-dataset", (1,), h5py.special_dtype(vlen=str))
meta_dataset = {'general': {'image_count': {'train': train_num, 'validate': validate_num, 'test': 0}, 
                            'labeled_image_count': train_num + validate_num, 
                            'unlabeled_image_count': 0, 
                            'image_dataset_format': "Binary", 
                            'description': '', 
                            'application_type': '', 
                            'data_created': '2020-11-16T10:32:46.003053', 
                            'project_token': '2ec1d738284daddacca94ca55ee652ec', 
                            'data_type': [], 
                            'module_name': [], 
                            'name': 'icdar2015', 
                            'consensus': {}}, 
                'location': {'bu': [], 
                             'campus': [], 
                             'plant': [], 
                             'line': [], 
                             'station': []}, 
                'image': {'color_type': 'BGR888', 
                          'image_resolution': {}}, 
                'manufacturing': {'product_name': [], 
                                  'product_sku': [], 
                                  'part_id': [], 
                                  'part_number': []}, 
                'label': {'categories': [{'id': 1, 
                                          'name': 'text', 
                                          'supercategory': ''}], 
                          'number_of_keypoints': None, 
                          'keypoint_name_list': None, 
                          'bar_code_type': None, 
                          'segmentation_type': 'segmentation', 
                          'label_type': ['polygon', 'text']}, 
                'filter_options': {}, 
                'misc': {'job_id': 1206, 
                         'job_name': 'local_build', 
                         'creator': 'wade_li', 
                         'resize': 'none'}}
h5["meta-dataset"][0] = json.dumps(meta_dataset)

   
    
for tag in dataset:
    counter = 0
    annotation_cointer = 0 
    print(tag)
    if tag == 'train':
        img_num = train_num
        img_path = 'train_images'
        txt_path = 'train_gts'
    else:
        img_num = validate_num
        img_path = 'test_images' 
        txt_path = 'test_gts'
    
    grp = h5.create_group(tag) 
    grp.create_dataset("images", (img_num,), dtype = h5py.special_dtype(vlen=np.dtype('uint8')))
    grp.create_dataset("meta-info", (img_num,), dt)
    grp.create_dataset("meta-label", (img_num,), dt)
    
    img_files = os.listdir(img_path)
    
    for img_file in img_files:
        if not 'img_102' in img_file:
            continue
        print(img_file)
        img = cv2.imread(os.path.join(img_path, img_file))
        success, encoded_image = cv2.imencode('.jpg', img)
        content = encoded_image.tobytes()
        h5[tag]["images"][counter] = np.fromstring(content, dtype='uint8')
        
        h5[tag]["meta-info"][counter] = json.dumps({"width": width,
                                                    "height": height, 
                                                    "color": "Binary", 
                                                    "file_name": img_file, 
                                                    "file_url": ''})
        if tag == 'train':
            txt_file = os.path.join(txt_path, img_file+'.txt')
        else:
            txt_file = os.path.join(txt_path, 'gt_'+img_file.strip('.jpg')+'.txt')
        meta_label_list = []

        f = open(txt_file, 'r', encoding='utf-8')
        line = f.readline()
        while line:
            txt = line.encode('utf-8').decode('utf-8-sig')
            txt = txt.split(',')
            #print(txt)
                
            x_list = []
            y_list = []
            seg_list = []           
            points = txt[0:8]
            point_num = len(points)/2 
            contour = []
               
            for point in range(0, int(point_num)+3, 2):
                coordinate = []
                x = float(points[point])
                y = float(points[point+1])
                x_list.append(x)
                y_list.append(y)
                seg_list.append(x)
                seg_list.append(y)
                    
                coordinate.append(int(x))
                coordinate.append(int(y))
                contour.append([coordinate])
            area = cv2.contourArea(np.array(contour))
            max_x = round(max(x_list))
            min_x = round(min(x_list))
                
            max_y = round(max(y_list))
            min_y = round(min(y_list))
            assert len(seg_list) == 8
            for i in seg_list:
                assert type(i) == float
            print(','.join(txt[8:]).strip('\n'))
            assert ','.join(txt[8:]).strip('\n') != ''
            meta_label_dict = {"category_id": 1,
                               "category": "text", 
                               "id": annotation_cointer, 
                               "image_id": counter, 
                               "iscrowd": 0, 
                               "segmentation": [seg_list], 
                               "area": area, 
                               "bbox": [float(min_x), float(min_y), max_x-min_x, max_y-min_y],
                               "text": ','.join(txt[8:]).strip('\n')
                              }
            meta_label_list.append(meta_label_dict)
            annotation_cointer += 1
            line = f.readline()
        h5[tag]["meta-label"][counter] = json.dumps(meta_label_list)   
        counter += 1
        
h5.close()

train
img_102.jpg
Shop
Dine
SMRT
validate
img_102.jpg
ST.MARC
CAFE
###
###


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:80: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


In [16]:
for i in seg_list:
    print(type(i))
    assert type(i) == float

<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>


In [6]:
f = h5py.File('test.h5','r')
print(f['validate'].keys())
i =0
print(f['meta-dataset'][0])
print(f['train']['meta-label'])
print(f['train']['images'][i])
print(f['train']['meta-label'][i])
print(f['train']['meta-info'][i])
print('========================================================')
print(f['validate']['images'][i])
print(f['validate']['meta-label'][i])
print(f['validate']['meta-info'][i])
f.close()

<KeysViewHDF5 ['images', 'meta-info', 'meta-label']>
{"general": {"image_count": {"train": 1, "validate": 1, "test": 0}, "labeled_image_count": 2, "unlabeled_image_count": 0, "image_dataset_format": "Binary", "description": "", "application_type": "", "data_created": "2020-11-16T10:32:46.003053", "project_token": "2ec1d738284daddacca94ca55ee652ec", "data_type": [], "module_name": [], "name": "icdar2015", "consensus": {}}, "location": {"bu": [], "campus": [], "plant": [], "line": [], "station": []}, "image": {"color_type": "BGR888", "image_resolution": {}}, "manufacturing": {"product_name": [], "product_sku": [], "part_id": [], "part_number": []}, "label": {"categories": [{"id": 1, "name": "text", "supercategory": ""}], "number_of_keypoints": null, "keypoint_name_list": null, "bar_code_type": null, "segmentation_type": "segmentation", "label_type": ["polygon", "text"]}, "filter_options": {}, "misc": {"job_id": 1206, "job_name": "local_build", "creator": "wade_li", "resize": "none"}}
<HD